In [ ]:
#Imports and paths setup
import sys
import rp
import torch
import numpy as np
from einops import rearrange
import shlex
from icecream import ic

top_dir = rp.get_git_toplevel()

ltx_dir = rp.path_join(top_dir, 'LTX2')
nfs_models_dir = rp.path_join(ltx_dir, 'models')

sys.path+=[
    nfs_models_dir,
]

from download_models import local_download_dir, download_from_web
local_models_dir = local_download_dir

models_dir = local_models_dir

In [ ]:
#Basic Post-Paths/Imports Setup
IN_NOTEBOOK = rp.running_in_jupyter_notebook()
DEVICE = rp.select_torch_device(prefer_used=True, reserve=True) #Select the GPU
DTYPE = torch.bfloat16

download_from_web() #Ensure sure base LTX models are downloaded

print(f"Running in notebook: {IN_NOTEBOOK}")
print(f"Top directory: {top_dir}")

In [ ]:
#CLAUDE: Modify this so the paths are corect. any paths must be declared in teh top cell. Any imports go to top cell too. 
#Make it T2V by the way.

from ltx_core.loader import LTXV_LORA_COMFY_RENAMING_MAP, LoraPathStrengthAndSDOps
from ltx_pipelines.ti2vid_two_stages import TI2VidTwoStagesPipeline

distilled_lora = [
    LoraPathStrengthAndSDOps(
        "/path/to/distilled_lora.safetensors",
        0.6,
        LTXV_LORA_COMFY_RENAMING_MAP
    ),
]

pipeline = TI2VidTwoStagesPipeline(
    checkpoint_path="/path/to/checkpoint.safetensors",
    distilled_lora=distilled_lora,
    spatial_upsampler_path="/path/to/upsampler.safetensors",
    gemma_root="/path/to/gemma",
    loras=[],
)

# Generate video from image
pipeline(
    prompt="A serene landscape with mountains in the background",
    output_path="output.mp4",
    seed=42,
    height=512,
    width=768,
    num_frames=121,
    frame_rate=25.0,
    num_inference_steps=40,
    cfg_guidance_scale=3.0,
    images=[("input_image.jpg", 0, 1.0)],  # Image at frame 0, strength 1.0
)